# Introduction: 
Purpose: to get the data ready for later analysis
    
Data Description: 12 files of Sales data (12 months of Sales in 2019). Each dataframe of each data file has 6 columns:
- Order ID: 
- Product: product name that customer bought in each order
- Quantity Ordered: number of items that customer ordered for each product name
- Price Each: Price of each item that customer bought
- Order Date: The date and time when each order was placed
- Purchase Address: the customer address to deliver products to

Data Problem: null values, wrong data types, lack of neccessary columns (for further analysis)

Task: 
<li>1. Import libraries, read files and append 12 files
<li>2. Change data type
<li>3. Clean data
<li>4. Add necessary columns
<li>5. Save the full dataframe as the name "Sales2019"

# Task 1. Import libraries, read files and append them

In [2]:
# Get a list of file names in the folder we want to read
file_names = os.listdir('data')

# Create an empty dataframe, to store all the data from 12 files
df = pd.DataFrame()

# Access each file by using file name, then append them
for file_name in file_names:
    # Append 12 files and store them into the empty dataframe "df"
    df = pd.concat([df, pd.read_csv(f'data/{file_name}')])

# Display the dataframe with 5 top rows
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


# Task 2. Change data types

In [3]:
# Change 'Order ID' to string
df.loc[:, 'Order ID'] = df['Order ID'].astype(str)

# Change 'Product' to string
df.loc[:, 'Product'] = df['Product'].astype(str)

# Change 'Quantity Ordered' to numeric
df.loc[:, 'Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'], errors='coerce')

# Change 'Price Each' to numeric
df.loc[:, 'Price Each'] = pd.to_numeric(df['Price Each'], errors='coerce')

# We will change 'Order Date' to datetime LATER, since there are wrong format values within the column

# Change 'Purchase Address' to string
df.loc[:, 'Purchase Address'] = df['Purchase Address'].astype(str)


"""
# Ignore the SettingWithCopyWarning 
# This code snippet is in case the CopyWarning appreared even after I fixed codes based on the instruction
import matplotlib.ticker as ticker  
from pandas.core.common import SettingWithCopyWarning  
import warnings
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
"""

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 186850 entries, 0 to 11685
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Order ID          186850 non-null  object 
 1   Product           186850 non-null  object 
 2   Quantity Ordered  185950 non-null  float64
 3   Price Each        185950 non-null  float64
 4   Order Date        186305 non-null  object 
 5   Purchase Address  186850 non-null  object 
dtypes: float64(2), object(4)
memory usage: 10.0+ MB


# 3. Clean data

In [4]:
# Display the number of null values
df.isna().sum()

Order ID              0
Product               0
Quantity Ordered    900
Price Each          900
Order Date          545
Purchase Address      0
dtype: int64

In [5]:
# Display 900 rows where Quantity Ordered values are empty
df[df['Quantity Ordered'].isnull()]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
664,nan,nan,NaN,NaN,NaN,nan
678,nan,nan,NaN,NaN,NaN,nan
797,nan,nan,NaN,NaN,NaN,nan
876,nan,nan,NaN,NaN,NaN,nan
1073,Order ID,Product,NaN,NaN,Order Date,Purchase Address
...,...,...,...,...,...,...
11384,nan,nan,NaN,NaN,NaN,nan
11399,Order ID,Product,NaN,NaN,Order Date,Purchase Address
11468,Order ID,Product,NaN,NaN,Order Date,Purchase Address
11574,Order ID,Product,NaN,NaN,Order Date,Purchase Address


In [6]:
# Drop null values ( 0.48% of data )
df = df.dropna()

In [7]:
# Check if there is any null values left
df.isnull().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

In [8]:
# Change 'Order Date' to datetime
df.loc[:, 'Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%y %H:%M')

# Check their data types once again
df.dtypes

Order ID                    object
Product                     object
Quantity Ordered           float64
Price Each                 float64
Order Date          datetime64[ns]
Purchase Address            object
dtype: object

In [9]:
# 99.5% data left with propriate data types
# Let's reset the index (to let it rearrange index after dropping some rows)
df = df.reset_index()

# Task 3. Add necessary columns (for further analysis)

In [13]:
# PRODUCT CATEGORY  COLUMN: 
# Split the values in the 'Product' column by whitespace and extract the last word
df['Product Category'] = df['Product'].str.split().str[-1]
# Replace "(4-pack)" to "Batteries" and replace 'iPhone' to 'Phone', since they are the last words
df.loc[df['Product Category']=='(4-pack)', 'Product Category'] = 'Batteries'
df.loc[df['Product Category']=='iPhone', 'Product Category'] = 'Phone'

# CITY AND STATE COLUMN:
# Split the values in the 'Purchase Address' column 
df['City'] = df['Purchase Address'].str.split(',').str[-2]
df['State'] = df['Purchase Address'].apply(lambda x: x.split(',')[2].split()[0])

# REVENUE COLUMN: 
df['Revenue'] = df['Quantity Ordered']*df['Price Each']

# MONTH COLUMN: 
df['Month'] = df['Order Date'].dt.month

# HOUR COLUMN
# Extract the hour from 'Order Date'
df['Hour'] = df['Order Date'].dt.hour

# DATE OF WEEK COLUMN:
# Extract the day of the week from 'Order Date'
df['DayOfWeek'] = df['Order Date'].dt.day_name()

# DATE TYPE COLUMN:
# Assign 'Weekday' or 'Weekend' based on the day of the week
df['DayType'] = df['DayOfWeek'].apply(lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday')

# Display the updated DataFrame with 5 top rows
df.head()

,index,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Product Category,City,State,Revenue,Month,Hour,DayOfWeek,DayType
0,0,141234,iPhone,1.0,700.00,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",Phone,Boston,MA,700.00,1,21,Tuesday,Weekday
1,1,141235,Lightning Charging Cable,1.0,14.95,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",Cable,Portland,OR,14.95,1,14,Monday,Weekday
2,2,141236,Wired Headphones,2.0,11.99,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",Headphones,San Francisco,CA,23.98,1,13,Thursday,Weekday
3,3,141237,27in FHD Monitor,1.0,149.99,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",Monitor,Los Angeles,CA,149.99,1,20,Saturday,Weekend
4,4,141238,Wired Headphones,1.0,11.99,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",Headphones,Austin,TX,11.99,1,11,Friday,Weekday


### For better understanding, I will replace all the abbreviated state names in State column by their full state names

In [21]:
state_mapping = {
    'CA': 'California',
    'GA': 'Georgia',
    'MA': 'Massachusetts',
    'ME': 'Maine',
    'NY': 'New York',
    'OR': 'Oregon',
    'TX': 'Texas',
    'WA': 'Washington'
}

df['State'] = df['State'].map(state_mapping)

# Task 5. Save the cleaned dataset with the name "Sales2019" for later use

In [19]:
# Save the clean dataset for later analysis
#df.to_csv('Sales2019', index = False)

#### *Upcoming: Exploratory Data Analysis (with charts and pointing out insights)